<a href="https://colab.research.google.com/github/saksham-42/Summer-Analytics-CAC-IITG/blob/Project/Capstone_Proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [217]:
from google.colab import files
uploaded = files.upload()
df = pd.read_csv("Data.csv")

Saving Data.csv to Data.csv
⏳ Waiting for streamed_prices.json...
⏳ Waiting for streamed_prices.json...
Waiting for streamed_prices.json...
Waiting for stream...
⏳ Waiting for stream... File streamed_prices.json does not exist
⏳ Waiting for data file...
⏳ Waiting for streamed_prices.json...
Error updating data: Expected object or value
⏳ Waiting for data... (Expected object or value)
⏳ Waiting for streamed_prices.json...
⏳ Waiting for data file...
Update error: [Errno 2] No such file or directory: 'streamed_prices.jsonl'
Error updating data: Expected object or value
⏳ Waiting for streamed_prices.json...
Error updating data: Expected object or value
Error updating data: Expected object or value


In [218]:
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,4/10/2016,7:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,4/10/2016,8:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,4/10/2016,8:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,4/10/2016,9:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,4/10/2016,9:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


⏳ Waiting for streamed_prices.json...
⏳ Waiting for streamed_prices.json...
Waiting for streamed_prices.json...
Waiting for stream...
⏳ Waiting for stream... File streamed_prices.json does not exist
⏳ Waiting for streamed_prices.json...
⏳ Waiting for streamed_prices.json...
⏳ Waiting for data file...
Error updating data: Expected object or value
⏳ Waiting for data... (Expected object or value)
⏳ Waiting for streamed_prices.json...
⏳ Waiting for data file...
Update error: [Errno 2] No such file or directory: 'streamed_prices.jsonl'
Error updating data: Expected object or value
Error updating data: Expected object or value
Error updating data: Expected object or value


Data Pre Processing + Feature Engineering

In [222]:
import pandas as pd
import numpy as np
from datetime import datetime
from math import radians, cos, sin, sqrt, atan2
from collections import defaultdict


# --- Fix & create timestamp column ---
import pandas as pd

# Load data
df = pd.read_csv("Data.csv")

# --- Step 1: Normalize and fill missing date and time ---
df["LastUpdatedDate"] = df["LastUpdatedDate"].fillna("01-01-1970").astype(str)
df["LastUpdatedTime"] = df["LastUpdatedTime"].fillna("00:00:00").astype(str)

# --- Step 2: Ensure date and time strings are valid ---
# Fix dates like "1/12/2016" -> "01-12-2016"
df["LastUpdatedDate"] = pd.to_datetime(df["LastUpdatedDate"], dayfirst=True, errors="coerce")
df["LastUpdatedDate"] = df["LastUpdatedDate"].dt.strftime("%d-%m-%Y")  # Standardize format

# Fix time to HH:MM:SS if malformed (e.g., 8:5 -> 08:05:00)
def fix_time(t):
    try:
        return pd.to_datetime(t, format="%H:%M:%S").strftime("%H:%M:%S")
    except:
        try:
            return pd.to_datetime(t, errors='coerce').strftime("%H:%M:%S")
        except:
            return "00:00:00"

df["LastUpdatedTime"] = df["LastUpdatedTime"].apply(fix_time)

# --- Step 3: Combine into timestamp ---
df["Timestamp"] = pd.to_datetime(
    df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
    format="%d-%m-%Y %H:%M:%S",
    errors="coerce"
)

# Drop rows with invalid timestamps
df = df.dropna(subset=["Timestamp"]).reset_index(drop=True)

# --- Ensure numeric columns ---
df["Capacity"] = pd.to_numeric(df["Capacity"], errors="coerce").fillna(1).replace(0, 1)
df["Occupancy"] = pd.to_numeric(df["Occupancy"], errors="coerce").fillna(0)
df["QueueLength"] = pd.to_numeric(df["QueueLength"], errors="coerce").fillna(0)
df["IsSpecialDay"] = pd.to_numeric(df["IsSpecialDay"], errors="coerce").fillna(0)

# --- Map vehicle types & traffic ---
vehicle_map = {"bike": 0.5, "car": 1.0, "truck": 1.5}
traffic_map = {"low": 0, "average": 1, "high": 2}

df = df[df["VehicleType"].isin(vehicle_map)]
df["VehicleWeight"] = df["VehicleType"].map(vehicle_map)

df["TrafficConditionNearby"] = df["TrafficConditionNearby"].where(
    df["TrafficConditionNearby"].isin(traffic_map), "average"
)
df["TrafficScore"] = df["TrafficConditionNearby"].map(traffic_map)

# --- Compute occupancy ratio safely ---
df["OccupancyRatio"] = df["Occupancy"] / df["Capacity"]

# --- Sort and reset index ---
df = df.sort_values(["SystemCodeNumber", "Timestamp"]).reset_index(drop=True)

# --- Data Processing Functions ---
def preprocess_data(filepath: str) -> pd.DataFrame:
    df = pd.read_csv(filepath)
    fill_values = {
        "Capacity": 1, "Occupancy": 0, "QueueLength": 0,
        "IsSpecialDay": 0, "VehicleType": "car", "TrafficConditionNearby": "average"
    }
    df.fillna(fill_values, inplace=True)
    df["Latitude"] = df["Latitude"].fillna(df["Latitude"].median())
    df["Longitude"] = df["Longitude"].fillna(df["Longitude"].median())
    return df

Model - 1

In [224]:
def apply_model1(group, base=10, alpha=5):
    prices = []
    last_price = base
    for occ_ratio in group["OccupancyRatio"]:
        price = np.clip(last_price + alpha * occ_ratio, 5, 20)
        prices.append(price)
        last_price = price
    group["Price_Model1"] = prices
    return group

df = df.groupby("SystemCodeNumber").apply(apply_model1).reset_index(drop=True)


⏳ Waiting for streamed_prices.json...
⏳ Waiting for streamed_prices.json...
Waiting for streamed_prices.json...
Waiting for stream...
⏳ Waiting for stream... File streamed_prices.json does not exist
⏳ Waiting for streamed_prices.json...
⏳ Waiting for streamed_prices.json...
⏳ Waiting for streamed_prices.json...
⏳ Waiting for data file...
Error updating data: Expected object or value
⏳ Waiting for data... (Expected object or value)
⏳ Waiting for data file...
Update error: [Errno 2] No such file or directory: 'streamed_prices.jsonl'
Error updating data: Expected object or value
Error updating data: Expected object or value
Error updating data: Expected object or value


Model - 2

In [225]:
A, B, C, D, E, LAMBDA = 4, 1, 1.5, 2, 1, 0.5

df["RawDemand"] = (
    A * df["OccupancyRatio"] +
    B * df["QueueLength"] -
    C * df["TrafficScore"] +
    D * df["IsSpecialDay"] +
    E * df["VehicleWeight"]
)

def normalize(group):
    min_d, max_d = group["RawDemand"].min(), group["RawDemand"].max()
    group["NormalizedDemand"] = (group["RawDemand"] - min_d) / (max_d - min_d + 1e-6)
    return group

df = df.groupby("SystemCodeNumber").apply(normalize).reset_index(drop=True)
df["Price_Model2"] = np.clip(10 * (1 + LAMBDA * df["NormalizedDemand"]), 5, 20)


⏳ Waiting for streamed_prices.json...
⏳ Waiting for streamed_prices.json...
Waiting for streamed_prices.json...
Waiting for stream...
⏳ Waiting for stream... File streamed_prices.json does not exist
⏳ Waiting for streamed_prices.json...
⏳ Waiting for streamed_prices.json...
⏳ Waiting for streamed_prices.json...
⏳ Waiting for data file...
Error updating data: Expected object or value
⏳ Waiting for data... (Expected object or value)
⏳ Waiting for data file...
Update error: [Errno 2] No such file or directory: 'streamed_prices.jsonl'
Error updating data: Expected object or value
Error updating data: Expected object or value
Error updating data: Expected object or value


Model - 3

In [226]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat, dlon = radians(lat2 - lat1), radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    return 2 * R * atan2(sqrt(a), sqrt(1 - a))

coords = df.groupby("SystemCodeNumber")[["Latitude", "Longitude"]].first()
nearby_lots = defaultdict(list)

for lot1, loc1 in coords.iterrows():
    for lot2, loc2 in coords.iterrows():
        if lot1 != lot2 and haversine(loc1["Latitude"], loc1["Longitude"], loc2["Latitude"], loc2["Longitude"]) <= 0.5:
            nearby_lots[lot1].append(lot2)

def apply_model3(df):
    price3 = []
    for _, row in df.iterrows():
        lot, ts, price2 = row["SystemCodeNumber"], row["Timestamp"], row["Price_Model2"]
        occ, cap = row["Occupancy"], row["Capacity"]
        comps = nearby_lots.get(lot, [])
        comp_prices = df[(df["SystemCodeNumber"].isin(comps)) & (df["Timestamp"] == ts)]["Price_Model2"].values
        if len(comp_prices):
            avg_comp = np.mean(comp_prices)
            if occ >= cap and avg_comp < price2:
                price2 -= 1
            elif avg_comp > price2:
                price2 += 0.5
        price3.append(np.clip(price2, 5, 20))
    df["Price_Model3"] = price3
    return df

df = apply_model3(df)


Pathway

In [277]:
# Re-import necessary libraries after kernel reset
import pandas as pd
import numpy as np
from datetime import datetime
from math import radians, cos, sin, sqrt, atan2
from collections import defaultdict
import json

# Load the dataset again
df = pd.read_csv("Data.csv")

# --- Fix & create timestamp column ---
df["LastUpdatedDate"] = df["LastUpdatedDate"].fillna("01-01-1970").astype(str)
df["LastUpdatedTime"] = df["LastUpdatedTime"].fillna("00:00:00").astype(str)
df["Timestamp"] = pd.to_datetime(
    df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
    dayfirst=True,
    errors="coerce"
)
df = df.dropna(subset=["Timestamp"])

# --- Ensure numeric columns ---
df["Capacity"] = pd.to_numeric(df["Capacity"], errors="coerce").fillna(1).replace(0, 1)
df["Occupancy"] = pd.to_numeric(df["Occupancy"], errors="coerce").fillna(0)
df["QueueLength"] = pd.to_numeric(df["QueueLength"], errors="coerce").fillna(0)
df["IsSpecialDay"] = pd.to_numeric(df["IsSpecialDay"], errors="coerce").fillna(0)

# --- Map vehicle types & traffic ---
vehicle_map = {"bike": 0.5, "car": 1.0, "truck": 1.5}
traffic_map = {"low": 0, "average": 1, "high": 2}
df = df[df["VehicleType"].isin(vehicle_map)]
df["VehicleWeight"] = df["VehicleType"].map(vehicle_map)
df["TrafficConditionNearby"] = df["TrafficConditionNearby"].where(
    df["TrafficConditionNearby"].isin(traffic_map), "average"
)
df["TrafficScore"] = df["TrafficConditionNearby"].map(traffic_map)
df["OccupancyRatio"] = df["Occupancy"] / df["Capacity"]
df = df.sort_values(["SystemCodeNumber", "Timestamp"]).reset_index(drop=True)

# --- Nearby mapping ---
coords = df.groupby("SystemCodeNumber")[["Latitude", "Longitude"]].first()
nearby_lots = defaultdict(list)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat, dlon = radians(lat2 - lat1), radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    return 2 * R * atan2(sqrt(a), sqrt(1 - a))
for lot1, loc1 in coords.iterrows():
    for lot2, loc2 in coords.iterrows():
        if lot1 != lot2 and haversine(loc1["Latitude"], loc1["Longitude"], loc2["Latitude"], loc2["Longitude"]) <= 0.5:
            nearby_lots[lot1].append(lot2)

# --- Model 1 ---
def apply_model1(group, base=10, alpha=5):
    prices = []
    last_price = base
    for occ_ratio in group["OccupancyRatio"]:
        price = np.clip(last_price + alpha * occ_ratio, 5, 20)
        prices.append(price)
        last_price = price
    group["Price_Model1"] = prices
    return group
df = df.groupby("SystemCodeNumber").apply(apply_model1).reset_index(drop=True)

# --- Model 2 ---
A, B, C, D, E, LAMBDA = 4, 1, 1.5, 2, 1, 0.5
df["RawDemand"] = (
    A * df["OccupancyRatio"] +
    B * df["QueueLength"] -
    C * df["TrafficScore"] +
    D * df["IsSpecialDay"] +
    E * df["VehicleWeight"]
)
def normalize(group):
    min_d, max_d = group["RawDemand"].min(), group["RawDemand"].max()
    group["NormalizedDemand"] = (group["RawDemand"] - min_d) / (max_d - min_d + 1e-6)
    return group
df = df.groupby("SystemCodeNumber").apply(normalize).reset_index(drop=True)
df["Price_Model2"] = np.clip(10 * (1 + LAMBDA * df["NormalizedDemand"]), 5, 20)

# --- Model 3 ---
def apply_model3(df):
    price3 = []
    for _, row in df.iterrows():
        lot, ts, price2 = row["SystemCodeNumber"], row["Timestamp"], row["Price_Model2"]
        occ, cap = row["Occupancy"], row["Capacity"]
        comps = nearby_lots.get(lot, [])
        comp_prices = df[(df["SystemCodeNumber"].isin(comps)) & (df["Timestamp"] == ts)]["Price_Model2"].values
        if len(comp_prices):
            avg_comp = np.mean(comp_prices)
            if occ >= cap and avg_comp < price2:
                price2 -= 1
            elif avg_comp > price2:
                price2 += 0.5
        price3.append(np.clip(price2, 5, 20))
    df["Price_Model3"] = price3
    return df
df = apply_model3(df)

# --- Select required columns and save as .jsonl ---
json_records = df[["Timestamp", "SystemCodeNumber", "Price_Model1", "Price_Model2", "Price_Model3"]].rename(
    columns={
        "SystemCodeNumber": "lot",
        "Price_Model1": "price1",
        "Price_Model2": "price2",
        "Price_Model3": "price3",
        "Timestamp": "timestamp"
    }
)
jsonl_path = "streamed_prices.jsonl"
with open(jsonl_path, "w") as f:
    for _, row in json_records.iterrows():
        row_dict = row.to_dict()
        row_dict["timestamp"] = row_dict["timestamp"].isoformat()
        f.write(json.dumps(row_dict) + "\n")

jsonl_path


Raw data loaded. Columns: ['timestamp', 'lot', 'price1', 'price2', 'price3']
Error updating data: 'Timestamp' object is not subscriptable
⏳ Waiting for data... ('SystemCodeNumber')
Update error: string indices must be integers, not 'str'
Raw data loaded. Columns: ['timestamp', 'lot', 'price1', 'price2', 'price3']
Error updating data: 'Timestamp' object is not subscriptable


'streamed_prices.jsonl'

Panel Dashboard

In [263]:
!pip install panel bokeh pandas --quiet

import pandas as pd
import panel as pn
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool
from datetime import datetime

pn.extension()

# --- Widgets & Plot Setup ---
dropdown = pn.widgets.Select(name='Parking Lot', options=[], width=250)
source = ColumnDataSource(data={
    'Timestamp': [],
    'base_price': [],
    'enhanced_price': [],
    'final_price': []
})

plot = figure(
    title="Real-Time Dynamic Parking Pricing Models",
    x_axis_type='datetime',
    width=900,
    height=450,
    tools="pan,wheel_zoom,box_zoom,reset"
)

# Plot pricing models
plot.line(x='Timestamp', y='base_price', source=source,
          legend_label='Base Price', color='blue', line_width=2)
plot.line(x='Timestamp', y='enhanced_price', source=source,
          legend_label='Enhanced Price', color='green', line_width=2)
plot.line(x='Timestamp', y='final_price', source=source,
          legend_label='Final Price', color='red', line_width=2)

plot.add_tools(HoverTool(
    tooltips=[
        ("Time", "@Timestamp{%F %T}"),
        ("Base", "@base_price{0.00}"),
        ("Enhanced", "@enhanced_price{0.00}"),
        ("Final", "@final_price{0.00}")
    ],
    formatters={"@Timestamp": "datetime"}
))
plot.legend.location = "top_left"
plot.legend.click_policy = "hide"

# --- Update Function ---
def update_data(event=None):
    try:
        df = pd.read_json("streamed_prices.jsonl", lines=True)
        if df.empty:
            return

        # --- Rename for consistency (match your Pathway output schema) ---
        df["Timestamp"] = pd.to_datetime(df["timestamp"])

        if not dropdown.options:
            lots = df["lot"].dropna().unique().tolist()
            dropdown.options = lots
            if lots:
                dropdown.value = lots[0]

        if dropdown.value:
            df_lot = df[df["lot"] == dropdown.value].copy()
            df_lot = df_lot.sort_values("Timestamp")

            # --- Extract nested price_info if present ---
            if "price_info" in df_lot.columns and df_lot["price_info"].notna().all():
                df_lot["base_price"] = df_lot["price_info"].apply(lambda x: x.get("base_price", 10.0))
                df_lot["enhanced_price"] = df_lot["price_info"].apply(lambda x: x.get("enhanced_price", 10.0))
                df_lot["final_price"] = df_lot["price_info"].apply(lambda x: x.get("final_price", 10.0))
            else:
                df_lot["base_price"] = df_lot["price1"]
                df_lot["enhanced_price"] = df_lot["price2"]
                df_lot["final_price"] = df_lot["price3"]

            source.data = {
                "Timestamp": df_lot["Timestamp"],
                "base_price": df_lot["base_price"],
                "enhanced_price": df_lot["enhanced_price"],
                "final_price": df_lot["final_price"]
            }

    except FileNotFoundError:
        print("Waiting")
    except Exception as e:
        print(" ")

# --- Initial Call + Live Refresh ---
update_data()
pn.state.add_periodic_callback(update_data, period=2000)  # every 2 seconds

# --- Layout ---
dashboard = pn.Column(
    "# Dynamic Parking Pricing Dashboard",
    pn.Row(
        dropdown,
        pn.widgets.StaticText(value="⏱ Updates every 2 seconds")
    ),
    plot,
    sizing_mode="stretch_width"
)

dashboard.servable()



Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] Row(sizing_mode='stretch_width')
        [0] Select(name='Parking Lot', options=['BHMBCCMKT01', ...], value='BHMBCCMKT01', width=250)
        [1] StaticText(sizing_mode='stretch_width', value='⏱ Updates every 2...)
    [2] Bokeh(figure, sizing_mode='stretch_width')

Update error: string indices must be integers, not 'str'
Raw data loaded. Columns: ['timestamp', 'lot', 'price1', 'price2', 'price3']
Error updating data: 'Timestamp' object is not subscriptable
Raw data loaded. Columns: ['timestamp', 'lot', 'price1', 'price2', 'price3']
Error updating data: 'Timestamp' object is not subscriptable
Raw data loaded. Columns: ['timestamp', 'lot', 'price1', 'price2', 'price3']
Error updating data: 'Timestamp' object is not subscriptable
⏳ Waiting for streamed_prices.json...
⏳ Waiting for streamed_prices.json...
Waiting for streamed_prices.json...
Waiting for stream...
⏳ Waiting for stream... File streamed_prices.json does not exist
⏳ Waiting for streamed_prices.json...
⏳ Waiting for streamed_prices.json...
⏳ Waiting for streamed_prices.json...
Error updating data: 'Timestamp' object is not subscriptable
Raw data loaded. Columns: ['timestamp', 'lot', 'price1', 'price2', 'price3']
Error updating data: 'Timestamp' object is not subscriptable
⏳ Waiting for str